# C interpreter

# First Stage

Design a grammar using an upward translation that accepts the following inputs from the C language:

- Assignment statements `(a = 5, a = b = c = 5)`
- Comparison operators `(==, <=, >=, !=)`
- Logical Operators `(&&, ||, !)`
- Arithmetic operators `(+, - , *, / and unary minus)`
- Variables and numerical constants `(a, 5)`



## Grammar design
First to design our grammar we have to now the precedence and the associativity of the operators.

### C operator precedence

The C operator predecence that we have to implement in our grammar is in this precedence order following [C operator Precedence](https://en.cppreference.com/w/c/language/operator_precedence) found in cppreference website:

1. Variables and numerical constants
2. Unary minus and Logical NOT
3. Arithmetic operators    
    - Multiplication, division, and reminder
    - addition and subtraction
4. Comparison operators
    - lt, gt, let, get
    - equals and not equals
4. Logical operators
    - Logical AND 
    - Logical OR
5.  Assignment statements

### Operator Associativity

In the same website we can see the operator associativity in summary:

#### Right to Left

- Logical NOT and unary minus
- Simple asisignment

#### Left to Right

- Arithmetic operator
- Comparison operators 
- Logical Operators 


### Tokens and non-terminal symbols

The non terminal symbols and tokens that we need to represent our language are:
- ID: to represent the variables `[a-zA-Z][\w_]*`
- CNUM: Numeric constants to represent the variable values or literal values, integer values for now.
- Operators: `[==, <=, >=, !=, &&, ||, !, +, - , *, / and unary minus]`

### Grammar design

- def -> asign `;`
- asign -> ID `=` asign
- asign -> expr
- expr -> exprOR
- exprOR -> exprOR `||` exprAND
- exprOR -> exprAND
- exprAND -> exprAND `&&` exprE
- exprAND -> exprE
- exprE -> exprE `[==, !=]` exprC
- exprE -> exprC
- exprC -> exprC `[<, <=, >, >=]` exprA
- exprC -> exprA
- exprA -> exprA `[+, -] ` add
- exprA -> add
- add -> add `[*, /]` fact
- add -> fact
- fact -> `-` fact
- fact -> num
- fact -> `ID`


## Requirements for implementation

- Install [sly](https://sly.readthedocs.io/en/latest/) library: We will use this library to implement the interpreter.
- Install [pandas](https://pandas.pydata.org/): We will use this library to import/represent data.

In [178]:
%%capture --no-display

!pip install sly
!pip install pandas

## Implementing the lexical analyzer (Lexer)

In [179]:
from sly import Lexer

class Scanner(Lexer):
    tokens = {ID, CNUM, EQ, NEQ, OR, AND, LEQ, GEQ}
    literals ={'=', '<', '>', '/', '!', '+', '-' , '*', ';'}

    # Ignore whitespace and tabulations

    ignore = ' \t'

    # Regular expressions rules for tokens

    ID = r'[a-zA-Z][\w_]*'
    EQ = r'=='
    NEQ = r'!='
    GEQ = r'>='
    LEQ = r'<='
    AND = r'&&'
    OR = r'\|\|'

    @_(r'\d+')
    def CNUM(self, t):
        t.value = int(t.value)
        return t

    # Error handling rule

    def error(self, t):
        print('<-'*10,"Illegal character '{}'".format(t.value[0]), '->'*10)
        self.index += 1
        t.type='Illegal'
        t.value =t.value[0]
        return t


## Testing the lexical analyzer (Lexer)


### Importing data

In [180]:
import pandas as pd 

data = pd.read_csv('../assets/testing/sentences.csv')
data[['sentences']]

,sentences
0,a = 5 + 5
1,5+5
2,5=5
3,=6
4,-5
5,c =-6
6,-9
7,a = b = c = d = r = 5 + 54
8,a = 5 - 54
9,a = -5


In [46]:
lexer = Scanner()
sentences = data['sentences'].values
pass_or_not = []
all_token_pass = True

for index, sentence in enumerate(sentences):
    print('-' * 80,"{} Lexically Testing sentence: '{}'".format(index, sentence),'-' * 80, sep='\n')
    for token in lexer.tokenize(sentence):
        print(" type = '{}', value = '{}'".format(token.type, token.value))
        if all_token_pass and 'Illegal' in token.type:
            all_token_pass = False
    
    pass_or_not.append('Pass') if all_token_pass else pass_or_not.append('FAIL')
    all_token_pass = True

data['Test'] = pass_or_not

--------------------------------------------------------------------------------
0 Lexically Testing sentence: 'a = 5 + 5'
--------------------------------------------------------------------------------
 type = 'ID', value = 'a'
 type = '=', value = '='
 type = 'CNUM', value = '5'
 type = '+', value = '+'
 type = 'CNUM', value = '5'
--------------------------------------------------------------------------------
1 Lexically Testing sentence: '5+5 '
--------------------------------------------------------------------------------
 type = 'CNUM', value = '5'
 type = '+', value = '+'
 type = 'CNUM', value = '5'
--------------------------------------------------------------------------------
2 Lexically Testing sentence: '5=5'
--------------------------------------------------------------------------------
 type = 'CNUM', value = '5'
 type = '=', value = '='
 type = 'CNUM', value = '5'
--------------------------------------------------------------------------------
3 Lexically Testing sent

In [5]:
data

,sentences,Test
0,a = 5 + 5,Pass
1,5+5,Pass
2,5=5,Pass
3,=6,Pass
4,-5,Pass
5,c =-6,Pass
6,-9,Pass
7,a = b = c = d = r = 5 + 54,Pass
8,a = 5 - 54,Pass
9,a = -5,Pass


## Implementing the Grammar and Semantic Analyzer (Parser)

In [173]:
from sly import Parser

class CInterpreterParser(Parser):
    tokens = Scanner.tokens

    def __init__(self):
        self.table = {}
    # - def -> asign `;`
    # - asign -> ID `=` asign
    # - asign -> expr
    # - expr -> exprOR
    # - exprOR -> exprOR `||` exprAND
    # - exprOR -> exprAND
    # - exprAND -> exprAND `&&` exprE
    # - exprAND -> exprE
    # - exprE -> exprE `[==, !=]` exprC
    # - exprE -> exprC
    # - exprC -> exprC `[<, <=, >, >=]` exprA
    # - exprC -> exprA
    # - exprA -> exprA `[+, -] ` add
    # - exprA -> add
    # - add -> add `[*, /]` fact
    # - add -> fact
    # - fact -> `-` fact
    # - fact -> num
    # - fact -> `ID`

    @_('asign ";"')
    def def_(self, p):
        pass
    @_('ID "=" asign')
    def asign(self, p):
        self.table[p.ID] = p.asign  
        return p.asign

    @_('expr')
    def asign(self, p):
        return p.expr

    @_('exprOR')
    def expr(self, p):
        return p.exprOR

    @_('exprOR OR exprAND')
    def exprOR(self, p):
        return bool(p.exprOR) | bool(p.exprAND)

    @_('exprAND')
    def exprOR(self, p):
        return p.exprAND

    @_('exprAND AND exprE')
    def exprAND(self, p):
        return bool(p.exprAND) & bool(p.exprE)

    @_('exprE') 
    def exprAND(self, p):
        return p.exprE

    @_('exprE NEQ exprC')
    def exprE(self, p):
        return p.exprE != p.exprC

    @_('exprE EQ exprC')
    def exprE(self, p):
        return p.exprE == p.exprC

    @_('exprC')
    def exprE(self, p):
        print(p.exprC)
        return p.exprC
    
    @_('exprC GEQ exprA')
    def exprC(self, p):
        return bool(p.exprC) < bool(p.exprA)

    @_('exprC ">" exprA')
    def exprC(self, p):
        return bool(p.exprC) > bool(p.exprA)

    @_('exprC LEQ exprA')
    def exprC(self, p):
        return bool(p.exprC) <= bool(p.exprA)

    @_('exprC "<" exprA')
    def exprC(self, p):
        return bool(p.exprC) < bool(p.exprA)

    @_('exprA')
    def exprC(self, p):
        return p.exprA

    @_('exprA "-" add')
    def exprA(self, p):
        return p.exprA - p.add
    
    @_('exprA "+" add')
    def exprA(self, p):
        return p.exprA + p.add

    @_('add')
    def exprA(self, p):
        return p.add

    @_('add "/" fact')
    def add(self, p):
        return p.add / p.fact

    @_('add "*" fact')
    def add(self, p):
        return p.add * p.fact

    @_('fact')
    def add(self, p):
        return p.fact

    @_('"-" fact')
    def fact(self, p):
        return -p.fact

    @_('CNUM')
    def fact(self, p):
        return p.CNUM

    @_('ID')
    def fact(self, p):
        try:
            return self.table[p.ID]
        except LookupError:
            print("Undefined name {}".format(p.ID))
            return 0


In [176]:
if __name__ == '__main__':
    lexer = Scanner()
    parser = CInterpreterParser()
    string = "b = c = r = 2 + -67;"
    parser.parse(lexer.tokenize(string))

-65


In [177]:
parser.table

{'r': -65, 'c': -65, 'b': -65}